In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
train_data =pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_data =pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
submission_data =pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')

In [3]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
train_data.shape


(1460, 81)

In [5]:

test_data.shape

(1459, 80)

## Data Cleaning


In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [7]:
# Alley
# Grvl	Gravel
#        Pave	Paved
#        NA 	No alley access
train_data['Alley'].value_counts()
# Replace NA with No_Alley


Grvl    50
Pave    41
Name: Alley, dtype: int64

In [8]:
train_data['Alley'] =  train_data['Alley'].replace(np.nan,'No_Alley')

In [9]:
# test_data['Alley'].value_counts()
test_data['Alley'] =  test_data['Alley'].replace(np.nan,'No_Alley')

In [10]:
# MasVnrType
# BrkCmn	Brick Common
#        BrkFace	Brick Face
#        CBlock	Cinder Block
#        None	None
#        Stone	Stone
train_data['MasVnrType'].value_counts()

None       864
BrkFace    445
Stone      128
BrkCmn      15
Name: MasVnrType, dtype: int64

In [11]:
# Replace 'NULL' with BrFace;mode
train_data['MasVnrType'] =  train_data['MasVnrType'].replace(np.nan,'None')

In [12]:
train_data['MasVnrType'].isnull().sum()

0

In [13]:
test_data['MasVnrType'].value_counts()

None       878
BrkFace    434
Stone      121
BrkCmn      10
Name: MasVnrType, dtype: int64

In [14]:
test_data['MasVnrType'].isnull().sum()

16

In [15]:
test_data['MasVnrType'] =  test_data['MasVnrType'].replace(np.nan,'None')

In [16]:
# MasVnrArea
# No of row with Null values
train_data['MasVnrArea'].isnull().sum()

8

In [17]:
# Find the mean of  MasVnrArea in train_data set
MasVnrArea_mean = train_data['MasVnrArea'].mean()

# Replace empty cells with 'MasVnrArea_mean'
train_data['MasVnrArea'] =  train_data['MasVnrArea'].replace(np.nan,MasVnrArea_mean)


In [18]:
# Find the mean of  MasVnrArea in test_data set
test_data['MasVnrArea'].isnull().sum()

15

In [19]:
MasVnrArea_meant = test_data['MasVnrArea'].mean()
# Replace empty cells with 'MasVnrArea_mean'
test_data['MasVnrArea'] =  test_data['MasVnrArea'].replace(np.nan,MasVnrArea_meant)

In [20]:
# BsmtQual
# Ex	Excellent (100+ inches)	
#        Gd	Good (90-99 inches)
#        TA	Typical (80-89 inches)
#        Fa	Fair (70-79 inches)
#        Po	Poor (<70 inches
#        NA	No Basement
train_data['BsmtQual'].isnull().sum()

37

In [21]:
# 37 null cells
train_data['BsmtQual'].value_counts()

TA    649
Gd    618
Ex    121
Fa     35
Name: BsmtQual, dtype: int64

In [22]:
# Replace 37 null cells with TA:
train_data['BsmtQual'] = train_data['BsmtQual'].replace(np.nan,'TA')

In [23]:
# test_data['BsmtQual'].isnull().sum()
# 44 null cells
test_data['BsmtQual'].value_counts()

TA    634
Gd    591
Ex    137
Fa     53
Name: BsmtQual, dtype: int64

In [24]:
train_data['BsmtQual'] = train_data['BsmtQual'].replace(np.nan,'TA')

In [25]:
# no missing value in train_data dataset

In [26]:
test_data.shape

(1459, 80)

In [27]:
train_data.shape

(1460, 81)

In [28]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          1459 non-null   object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [29]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          1460 non-null   object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [30]:
# GarageType
# GarageType: Garage location
		
#        2Types	More than one type of garage
#        Attchd	Attached to home
#        Basment	Basement Garage
#        BuiltIn	Built-In (Garage part of house - typically has room above garage)
#        CarPort	Car Port
#        Detchd	Detached from home
#        NA	No Garage
# 76 empty in test set
# 81 empty in train set
train_data['GarageType'].isnull().sum()

81

In [31]:
test_data['GarageType'].value_counts()

Attchd     853
Detchd     392
BuiltIn     98
Basment     17
2Types      17
CarPort      6
Name: GarageType, dtype: int64

In [32]:
train_data['GarageType'] = train_data['GarageType'].replace(np.nan,'Attchd')
test_data['GarageType'] = test_data['GarageType'].replace(np.nan,'Attchd')

In [33]:
import pandas as pd

# function that replace empty cells with mode of the column value
def replace_empty(df):
    for column in df.columns:
        if df[column].dtype == 'object':
            mode = df[column].mode()[0]
            df[column].fillna(mode, inplace=True)
        else:
            df[column] = pd.to_numeric(df[column], errors='coerce')
            mode = df[column].mode()[0]
            df[column].fillna(mode, inplace=True)
    return df

In [34]:
train_data =replace_empty(train_data)

In [35]:
test_data= replace_empty(test_data)

In [36]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1459 non-null   object 
 3   LotFrontage    1459 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          1459 non-null   object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1459 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [37]:
X = train_data.copy()
y = X.pop("SalePrice")

# Label encoding for categoricals
for colname in X.select_dtypes("object"):
    X[colname], _ = X[colname].factorize()

# All discrete features should now have integer dtypes (double-check this before using MI!)
discrete_features = X.dtypes == int

from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
mi_scores[::3]  # show a few features with their MI scores

ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.

In [ ]:
# FireplaceQu
# FireplaceQu: Fireplace quality

#        Ex	Excellent - Exceptional Masonry Fireplace
#        Gd	Good - Masonry Fireplace in main level
#        TA	Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement
#        Fa	Fair - Prefabricated Fireplace in basement
#        Po	Poor - Ben Franklin Stove
#        NA	No Fireplace

#690 empty in train_data set
# 730 empty in test_data set
test_data['FireplaceQu'].isnull().sum()

In [ ]:
# train_data['FireplaceQu'].value_counts()
train_data['FireplaceQu'] = train_data['FireplaceQu'].replace(np.nan,'Gd')
test_data['FireplaceQu'] = test_data['FireplaceQu'].replace(np.nan,'Gd')

In [ ]:
test_data['FireplaceQu'].value_counts()

In [ ]:
# Electrical

# Electrical: Electrical system

#        SBrkr	Standard Circuit Breakers & Romex
#        FuseA	Fuse Box over 60 AMP and all Romex wiring (Average)	
#        FuseF	60 AMP Fuse Box and mostly Romex wiring (Fair)
#        FuseP	60 AMP Fuse Box and mostly knob & tube wiring (poor)
#        Mix	Mixed

# 1 empty in train_data
# train_data['Electrical'].isnull().sum()
train_data['Electrical'].value_counts()

In [ ]:
train_data['Electrical'] = train_data['Electrical'].replace(np.nan,'SBrkr')

In [ ]:
# 37 null columns
# BsmtCond: Evaluates the general condition of the basement

#        Ex	Excellent
#        Gd	Good
#        TA	Typical - slight dampness allowed
#        Fa	Fair - dampness or some cracking or settling
#        Po	Poor - Severe cracking, settling, or wetness
#        NA	No Basement

train_data['BsmtCond'].isnull().sum()

In [ ]:
# 45 with null entry
test_data['BsmtCond'].isnull().sum()

In [ ]:
train_data['BsmtCond'].value_counts()

In [ ]:
# Replace NA with TA:-- Slight dampness allowed
train_data['BsmtCond'] = train_data['BsmtCond'].replace(np.nan,'TA')
test_data['BsmtCond'] = test_data['BsmtCond'].replace(np.nan,'TA')

In [ ]:
# BsmtExposure
# 38 empty
# BsmtExposure: Refers to walkout or garden level walls

#        Gd	Good Exposure
#        Av	Average Exposure (split levels or foyers typically score average or above)	
#        Mn	Mimimum Exposure
#        No	No Exposure
#        NA	No Basement

train_data['BsmtExposure'].isnull().sum()

In [ ]:
train_data['BsmtExposure'].value_counts()

In [ ]:
test_data['BsmtExposure'].value_counts()

In [ ]:
train_data['BsmtExposure'] = train_data['BsmtExposure'].replace(np.nan,'No')
test_data['BsmtExposure'] = test_data['BsmtExposure'].replace(np.nan,'No')

In [ ]:
# BsmtFinType1
# BsmtFinType1: Rating of basement finished area

#        GLQ	Good Living Quarters
#        ALQ	Average Living Quarters
#        BLQ	Below Average Living Quarters	
#        Rec	Average Rec Room
#        LwQ	Low Quality
#        Unf	Unfinshed
#        NA	No Basement

# train_data['BsmtFinType1'].value_counts()
train_data['BsmtFinType1'] = train_data['BsmtFinType1'].replace(np.nan,'Unf')
test_data['BsmtFinType1'] = test_data['BsmtFinType1'].replace(np.nan,'GLQ')

In [ ]:
# Extracting some features from the dataset
features = ['OverallQual','BsmtQual','GarageArea','YearBuilt','YearRemodAdd','Fireplaces','MSZoning','MasVnrType','LotArea',
       'SaleCondition','SaleType','Exterior2nd','GarageYrBlt','MSSubClass'
                    ]
train_data = train_data[features]
train_data.head()